In [2]:
!pip install -q datasets

In [10]:
import pandas as pd
from datasets import load_dataset
paws_dataset = load_dataset('paws', 'labeled_final')
train_df = pd.DataFrame({'input_text': paws_dataset['train']['sentence1'], 'target_text': paws_dataset['train']['sentence2'], 'label': paws_dataset['train']['label']})
test_df = pd.DataFrame({'input_text': paws_dataset['test']['sentence1'], 'target_text': paws_dataset['test']['sentence2'], 'label': paws_dataset['test']['label']})
df = pd.concat([train_df, test_df], ignore_index=True)


  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
df = df[df['label']==1]
df = df.reset_index(drop=True)
df = df.drop(columns=['label'])
df.head()

,input_text,target_text
0,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...
1,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...
2,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...
3,William Henry Henry Harman was born on 17 Febr...,"William Henry Harman was born in Waynesboro , ..."
4,With a discrete amount of probabilities Formul...,Given a discrete set of probabilities formula ...


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25365 entries, 0 to 25364
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   input_text   25365 non-null  object
 1   target_text  25365 non-null  object
dtypes: object(2)
memory usage: 396.5+ KB


In [19]:
from transformers import pipeline
import tensorflow as tf
model = pipeline('text-classification', model='SamLowe/roberta-base-go_emotions', top_k=None)
positive_grad=['amusement','excitement','pride','optimism','gratitude','joy','admiration','love','approval','caring','desire','relief']
neutral_grad=['confusion','curiosity','realization','surprise','neutral']
negative_grad=['anger','disgust','grief','fear','sadness','nervousness','annoyance','disappointment','embarrassment','remorse','disapproval']
total_grad=positive_grad+neutral_grad+negative_grad

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [20]:
df.head()

,input_text,target_text
0,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...
1,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...
2,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...
3,William Henry Henry Harman was born on 17 Febr...,"William Henry Harman was born in Waynesboro , ..."
4,With a discrete amount of probabilities Formul...,Given a discrete set of probabilities formula ...


In [25]:
def predict_emotion(text):
    model_output = model([text])
    predictions = model_output[0]
    
    max_score_index = max(range(len(predictions)), key=lambda i: predictions[i]["score"])
    if max_score_index<0.5:
        return 'neutral'
    else:
        return predictions[max_score_index]["label"]

In [22]:
from tqdm import tqdm

for index, row in tqdm(df.iterrows(), total=len(df), desc='Processing rows'):
    input_text = row['input_text']
    target_text = row['target_text']

    input_emotion = predict_emotion(input_text)
    target_emotion = predict_emotion(target_text)

    if (input_emotion == target_emotion or input_emotion == 'neutral' or target_emotion == 'neutral'):
        df.drop(index, inplace=True)
    else:
        if total_grad.index(input_emotion) > total_grad.index(target_emotion):
            df.at[index, 'input_text'], df.at[index, 'target_text'] = df.at[index, 'target_text'], df.at[index, 'input_text']
            input_emotion, target_emotion = target_emotion, input_emotion
        emotion_transition = f"{input_emotion} to {target_emotion}: "
        modified_input_text = emotion_transition + input_text
        df.at[index, 'input_text'] = modified_input_text

Processing rows: 100%|██████████| 25365/25365 [54:35<00:00,  7.74it/s] 


In [54]:
df.to_csv('paws_3.csv',index=False)

In [23]:
df.head(5)

,input_text,target_text
1118,sadness to disappointment: After the Muslim in...,After the Muslim invasions of the seventh cent...
1534,approval to realization: With the weakening of...,"With the weakening of the Canadian dollar , ma..."
5642,admiration to approval: The album received a m...,The album received a largely positive critical...
6021,"admiration to approval: In this way , it was p...",In this way it was possible to literally recor...
6148,approval to realization: With the weakening of...,"With the weakening of the Canadian dollar , ma..."


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22 entries, 1118 to 25051
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   input_text   22 non-null     object
 1   target_text  22 non-null     object
dtypes: object(2)
memory usage: 528.0+ bytes
